In [ ]:
import pandas as pd
import glob


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# récupérer les fichier du dossier contenant les fihcier de clusters
# glob aide à faire un trie par regex afin d'en choisir les fichiers nettoyés
list_brut= glob.glob('/content/drive/MyDrive/Colab Notebooks/km/*raw*')
list_cluster= glob.glob('/content/drive/MyDrive/Colab Notebooks/km/*clean*')

In [ ]:
# vérifier la liste
print(len(list_cluster))

10


In [ ]:
list_brut.sort()
list_cluster.sort()

In [ ]:
for i in range(10):
  print(list_brut[i])

In [ ]:
for i in range(10):
  print(list_cluster[i])

In [ ]:
# fonction pour lire une liste de fichier 
# argument : une liste de fichiers
# sortie: dictionnaire
def read_cluster(files):
  liste=[]
  for i,cluster in enumerate(files):
    with open(cluster, "r") as file:
      liste.append(file.readlines())
  return liste

In [ ]:
clean_cluster= read_cluster(list_cluster)

In [ ]:
brut_cluster= read_cluster(list_brut)

In [ ]:
print((clean_cluster[1]))

['roser\n', 'our choice is rosé mamavote roser\n', 'ra_apex_divisioncommission for\n', 'manifesting surprise dm manifesting surprise dm manifesting surprise dm manifesting surprise dm manifesting surprise dm manifesting surprise dm manifesting surprise dm manifesting surprise dm\n', 'vote\n', 'vot\n', 'vote carats\n', 'géministhesong butter pcer\n', 'votes votes votes vote votes vote come elf vote superjunior for mamavote\n', 'themusicvideo butter pca vote bts\n', 'vote damn\n', 'vote\n', 'vote\n', 'vote\n', 'vote\n', 'votestotal vote\n', 'manifesting surprise dm\n', 'more votes for enhypen\n', 'vot enhypen\n', 'mama and mma vote mamavote roser\n', 'rosé is my choic mamavote roser\n', 'vote mamavote sf\n', 'vot vote somi mamavote sommungchis vot somi top linkvotemamavote\n', 'our choice is rosémamavote roser\n', 'vote mamavote sf\n', 'vote mamavote sf\n', 'vote rizobwm\n', 'vot for rosé mamavote roser\n', 'vote\n', 'vote\n', 'our choice is rosémamavote roser\n', 'vot mamavote sf\n', 'f

In [ ]:
# Mettre le contenu de chaque cluster dans un dataframe
for i in range(len(brut_cluster)):
    vars()["data_"+str(i)] = pd.DataFrame(list(zip(brut_cluster[i] ,clean_cluster[i])), columns = ['tweet', 'clean_twt'])

In [ ]:
!pip install gensim==4.0
!pip install python-Levenshtein

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
# récupérer les embeddings entrainés sur nos données
w2v = KeyedVectors.load('/content/drive/MyDrive/Colab Notebooks/w2vec_model_d300_1M')

In [ ]:
import numpy as np

In [ ]:
#encode un message
def encode(msg):
    return np.mean([w2v[word] for word in msg if word in w2v] or [np.zeros(300)], axis = 0)

In [ ]:
w2v.similar_by_word("sale")

In [ ]:
# Encoder une liste de messages 
def encoded(liste):
  w2v=[]
  for i in range(len(liste)):
    w2v.append(np.array([encode(msg.split()) for msg in liste[i]]))
  return w2v

In [ ]:
%%time
#encodage de l'échantillon pour créer le jeu de données
dataset_encoded = encoded(clean_cluster)

CPU times: user 27.7 s, sys: 786 ms, total: 28.5 s
Wall time: 29.3 s


In [ ]:
from sklearn.ensemble import IsolationForest

# definir le modèle ainsi que ses paramètres
clf=IsolationForest(n_estimators=100, max_samples='auto', \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)


In [ ]:
# Entrainer le modèle
%%time
for i in range(len(dataset_encoded)):
  clf.fit(dataset_encoded[i])

CPU times: user 2min 57s, sys: 985 ms, total: 2min 58s
Wall time: 1min 33s


In [ ]:
#mettre les prédictions dans une liste
%%time
pred=[]
for i in range(len(dataset_encoded)):
  pred.append(clf.predict(dataset_encoded[i]))

CPU times: user 1min 2s, sys: 391 ms, total: 1min 3s
Wall time: 1min 3s


In [ ]:
len(pred[7])

47500

In [ ]:
# ajouter une colonne pred à chaque dataframe de cluster
for i in range(len(dataset_encoded)):
  vars()["data_"+str(i)]['pred']= pred[i]

In [ ]:
data_1.head(20)

In [ ]:
# Trouver les outliers de chaque cluster
# ceux dont l'index est -1
outliers=[]
for i in range(len(dataset_encoded)):
  outliers.append((vars()["data_"+str(i)]['tweet'].loc[vars()["data_"+str(i)]['pred']==-1]))
  outlier_index=list(outliers[i].index)
  print("cluster"+str(i)+":"+"\n")
  print(vars()["data_"+str(i)]['pred'].value_counts())
  print("\n")

cluster0:

 1    86797
-1     3185
Name: pred, dtype: int64


cluster1:

-1    3428
 1     755
Name: pred, dtype: int64


cluster2:

-1    2863
 1    1470
Name: pred, dtype: int64


cluster3:

 1    101245
-1      6026
Name: pred, dtype: int64


cluster4:

 1    177156
-1      7325
Name: pred, dtype: int64


cluster5:

1    31861
Name: pred, dtype: int64


cluster6:

 1    90354
-1     3146
Name: pred, dtype: int64


cluster7:

 1    44282
-1     3218
Name: pred, dtype: int64


cluster8:

 1    75924
-1     5201
Name: pred, dtype: int64


cluster9:

 1    12635
-1     1261
Name: pred, dtype: int64




In [ ]:
outliers[3]

33             0 dignité mais bon https://t.co/g6QuTq9Oyj\n
34                      0 dose et ça bougera pas pour moi\n
35                    0 éducation https://t.co/mzqz7Mw2rx\n
36                             0 en allemand rien compris\n
40                                    0 nul 0 défaite ???\n
                                ...                        
107231                               Z'êtes coupables ! 🤓\n
107237                       Zhoon c'est en quelle langue\n
107250                       Zozjzksjsb c en France ça???\n
107260                  Zvragevavzgbzgv Histoire Moderne?\n
107267    #パラライ幻影トレーラー解禁Double Trouble https://t.co/QjYK...
Name: tweet, Length: 6026, dtype: object

In [ ]:
for i in range(len(outliers)):
  with open(f"drive/MyDrive/outliers/outliers{i}", 'w') as f:
    for line in outliers[i]:
      f.write(line)